In [2]:
import pandas as pd
dataset=pd.read_csv('insurance_pre.csv')
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [3]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [4]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [5]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset[['charges']]

In [7]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
indep=scx.fit_transform(indep)
indep

array([[-1.43876426, -0.45332   , -0.90861367, -1.0105187 ,  1.97058663],
       [-1.50996545,  0.5096211 , -0.07876719,  0.98959079, -0.5074631 ],
       [-0.79795355,  0.38330685,  1.58092576,  0.98959079, -0.5074631 ],
       ...,
       [-1.50996545,  1.0148781 , -0.90861367, -1.0105187 , -0.5074631 ],
       [-1.29636188, -0.79781341, -0.90861367, -1.0105187 , -0.5074631 ],
       [ 1.55168573, -0.26138796, -0.90861367, -1.0105187 ,  1.97058663]])

In [10]:
scy=StandardScaler()
dep=scy.fit_transform(dep)
dep

array([[ 0.2985838 ],
       [-0.95368917],
       [-0.72867467],
       ...,
       [-0.96159623],
       [-0.93036151],
       [ 1.31105347]])

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
param_grid= {'fit_intercept':[True,False ], 'copy_X':[True,False], 'n_jobs':[1,-1], 'positive':[False,True]}
grid=GridSearchCV(LinearRegression(), param_grid, refit=True,verbose=3, n_jobs=-1)
grid.fit(indep, dep)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=LinearRegression(), n_jobs=-1,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False], 'n_jobs': [1, -1],
                         'positive': [False, True]},
             verbose=3)

In [12]:
grid.best_params_

{'copy_X': True, 'fit_intercept': False, 'n_jobs': 1, 'positive': True}

In [13]:
y_pred=grid.predict(indep)
y_pred

array([[ 1.01747291],
       [-0.77563818],
       [-0.50487876],
       ...,
       [-0.73286418],
       [-0.96275249],
       [ 1.94318926]])

In [15]:
from sklearn.metrics import r2_score
r2=r2_score(dep,y_pred)
print('R2 Score value for best parameter: {}'.format(grid.best_params_),  r2)

R2 Score value for best parameter: {'copy_X': True, 'fit_intercept': False, 'n_jobs': 1, 'positive': True} 0.749694530346479


In [16]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_copy_X,param_fit_intercept,param_n_jobs,param_positive,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.224725,0.112970,0.009948,0.003581,True,True,1,False,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.760979,0.708733,0.775584,0.731367,0.756545,0.746642,0.023713,13
1,0.043602,0.017526,0.007363,0.004642,True,True,1,True,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.761482,0.708782,0.775584,0.731353,0.756751,0.746790,0.023778,9
2,0.002166,0.004333,0.004274,0.005245,True,True,-1,False,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.760979,0.708733,0.775584,0.731367,0.756545,0.746642,0.023713,13
3,0.039379,0.008668,0.000000,0.000000,True,True,-1,True,"{'copy_X': True, 'fit_intercept': True, 'n_job...",0.761482,0.708782,0.775584,0.731353,0.756751,0.746790,0.023778,9
4,0.002460,0.003877,0.000458,0.000917,True,False,1,False,"{'copy_X': True, 'fit_intercept': False, 'n_jo...",0.761261,0.709084,0.776022,0.731370,0.756912,0.746930,0.023773,5
5,0.002486,0.004393,0.000000,0.000000,True,False,1,True,"{'copy_X': True, 'fit_intercept': False, 'n_jo...",0.761752,0.709114,0.776022,0.731355,0.757135,0.747076,0.023844,1
6,0.004049,0.004959,0.002001,0.004003,True,False,-1,False,"{'copy_X': True, 'fit_intercept': False, 'n_jo...",0.761261,0.709084,0.776022,0.731370,0.756912,0.746930,0.023773,5
7,0.023188,0.003660,0.000626,0.001252,True,False,-1,True,"{'copy_X': True, 'fit_intercept': False, 'n_jo...",0.761752,0.709114,0.776022,0.731355,0.757135,0.747076,0.023844,1
8,0.002639,0.003906,0.003321,0.004218,False,True,1,False,"{'copy_X': False, 'fit_intercept': True, 'n_jo...",0.760979,0.708733,0.775584,0.731367,0.756545,0.746642,0.023713,13
9,0.004930,0.004331,0.000000,0.000000,False,True,1,True,"{'copy_X': False, 'fit_intercept': True, 'n_jo...",0.761482,0.708782,0.775584,0.731353,0.756751,0.746790,0.023778,9
